In [24]:
! lamin load scprint

^C
Traceback (most recent call last):
  File "/home/ml4ig1/miniconda3/envs/scprint/bin/lamin", line 5, in <module>
    from lamin_cli.__main__ import main
  File "/home/ml4ig1/miniconda3/envs/scprint/lib/python3.10/site-packages/lamin_cli/__main__.py", line 11, in <module>
    import rich_click as click
  File "/home/ml4ig1/miniconda3/envs/scprint/lib/python3.10/site-packages/rich_click/__init__.py", line 73, in <module>
    from . import rich_click as rich_click
  File "/home/ml4ig1/miniconda3/envs/scprint/lib/python3.10/site-packages/rich_click/rich_click.py", line 6, in <module>
    import rich.columns
  File "/home/ml4ig1/miniconda3/envs/scprint/lib/python3.10/site-packages/rich/columns.py", line 7, in <module>
    from .console import Console, ConsoleOptions, RenderableType, RenderResult
  File "/home/ml4ig1/miniconda3/envs/scprint/lib/python3.10/site-packages/rich/console.py", line 57, in <module>
    from .markup import render as render_markup
  File "<frozen importlib._bootstra

In [1]:
from lightning.pytorch import Trainer, seed_everything
from lightning.pytorch.callbacks import ModelCheckpoint, StochasticWeightAveraging, EarlyStopping, LearningRateMonitor, LearningRateFinder

seed_everything(42, workers=True)

from scprint import scPrint
from scprint.trainer import TrainingMode
from scdataloader import DataModule 
import pandas as pd
from scdataloader.utils import load_genes

import torch
torch.set_float32_matmul_precision('medium')

%load_ext autoreload
%autoreload 2

Global seed set to 42


💡 connected lamindb: jkobject/scprint


2024-05-21 16:55:39,115:INFO - Downloading data from `https://omnipathdb.org/queries/enzsub?format=json`
2024-05-21 16:55:39,224:INFO - Downloading data from `https://omnipathdb.org/queries/interactions?format=json`
2024-05-21 16:55:39,315:INFO - Downloading data from `https://omnipathdb.org/queries/complexes?format=json`
2024-05-21 16:55:39,405:INFO - Downloading data from `https://omnipathdb.org/queries/annotations?format=json`
2024-05-21 16:55:39,502:INFO - Downloading data from `https://omnipathdb.org/queries/intercell?format=json`
2024-05-21 16:55:39,684:INFO - Downloading data from `https://omnipathdb.org/about?format=text`
/home/ml4ig1/miniconda3/envs/scprint/lib/python3.10/site-packages/umap/__init__.py:9: ImportWarning: Tensorflow not installed; ParametricUMAP will be unavailable
  warn(
/home/ml4ig1/miniconda3/envs/scprint/lib/python3.10/site-packages/chex/_src/pytypes.py:53: DeprecationWarning: jax.core.Shape is deprecated. Use Shape = Sequence[int | Any].
  Shape = jax.core

In [2]:
# TODO: drop tissue & dev stage until part or is taken in account

hierarchical_clss = [
    "cell_type_ontology_term_id",  # 1
    # "tissue_ontology_term_id",
    "disease_ontology_term_id",  # 2
    #    "development_stage_ontology_term_id",
    "assay_ontology_term_id",  # 3
    'self_reported_ethnicity_ontology_term_id',  # 4
]
clss_to_pred = hierarchical_clss+[
    'sex_ontology_term_id',  # 5
    "organism_ontology_term_id",  # 6
]
all_clss = clss_to_pred+[
    # 'dataset_id',
    # 'cell_culture',
    #  "heat_diff",
    #  "total_counts",
    # "nnz",
    #  "dpt_group",
]

gene_emb = '../data/main/gene_embeddings.parquet'
d_model = 128

In [3]:
datamodule = DataModule(
    collection_name="some",
    gene_embeddings=gene_emb,
    all_clss=all_clss,
    hierarchical_clss=hierarchical_clss,
    organisms=["NCBITaxon:9606"],  # , "NCBITaxon:10090"],
    how="most expr",
    max_len=1200,
    add_zero_genes=0,
    # how much more you will see the most present vs less present category
    weight_scaler=10,
    clss_to_weight=clss_to_pred,
    clss_to_pred=clss_to_pred,
    batch_size=1,
    num_workers=1,
    # train_oversampling=2,
    validation_split=0.05,
    do_gene_pos='../data/main/biomart_pos.parquet',
    test_split=0.05)
testfiles = datamodule.setup()

won't do any check but we recommend to have your dataset coming from local storage

80.0% are aligned
seeing a string: loading gene positions as biomart parquet file


In [4]:
# create a function to transform an scGPT checkpoint to an scPrint's
# ckpt = torch.load("../../scGPT/save/model_e6.pt")
# scPrint.load_from_checkpoint("../../scGPT/save/model_e6.pt")

In [5]:
# from lightning.pytorch.profilers import PyTorchProfiler
# pytorch_prof = PyTorchProfiler("../data/tensorboard", emit_nvtx=False, group_by_input_shape=True, record_shapes=True, profile_memory=True, with_stack=True, on_trace_ready=torch.profiler.tensorboard_trace_handler("../data/tensorboard/"),)

In [4]:
model = scPrint(
    genes=datamodule.genes,
    d_model=d_model*4,
    nhead=4*2,
    nlayers=1,
    # layers_cls = [d_model],
    # labels = datamodule.labels,
    # cls_hierarchy = datamodule.cls_hierarchy,
    dropout=0,
    transformer="flash",
    precpt_gene_emb=gene_emb,
    gene_pos_enc=datamodule.gene_pos,
    mvc_decoder="inner product",
    # label_decoders = datamodule.decoders,
    fused_dropout_add_ln=False,
    # num_batch_labels = datamodule.num_datasets,
    checkpointing=False,
    prenorm=True,
    num_heads_kv=None,
    weight_decay=0,
    #zinb=False
)

In [6]:
# from lightning.pytorch.loggers import TensorBoardLogger
from lightning.pytorch.loggers import WandbLogger

wandb_logger = WandbLogger(project="scprint_test",
                           save_dir="../data/tensorboard")
wandb_logger.watch(model, log='all', log_freq=50, log_graph=True)

# tlogger = TensorBoardLogger(save_dir="../data/tensorboard")
# tlogger.log_graph(model)

2024-05-21 16:00:36,256:ERROR - Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jkobject (ml4ig). Use `wandb login --relogin` to force relogin
/home/ml4ig1/miniconda3/envs/scprint/lib/python3.10/site-packages/wandb/sdk/lib/ipython.py:77: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import HTML, display  # type: ignore


wandb: logging graph, to disable use `wandb.watch(log_graph=False)`


In [7]:
chckp = ModelCheckpoint(monitor="val_loss", save_top_k=-1)
trainingmode = TrainingMode(
    do_denoise=True,
    noise=[0.01],
    do_cce=False,
    cce_sim=0.6,
    do_ecs=False,
    ecs_threshold=0.4,
    ecs_scale=0.05,
    class_scale=0.08,
    do_cls=False,
    do_mvc=False,
    do_adv_cls=False,
    do_next_tp=False,
    mask_ratio=[],
    warmup_duration=100,
    fused_adam=True,
    lr_reduce_patience=200,
)
# es = EarlyStopping(patience=2, monitor='val_loss')
# swa = StochasticWeightAveraging(swa_lrs= 0.01)
# lrm = LearningRateMonitor(logging_interval="step")
# lrf = LearningRateFinder(mode="exponential",)
# TODO: to check that the class hierarchy are really ordered from 1-2-3-4... as well (oredered dict)
# , logger=tlogger) #detect_anomaly=True, fast_dev_run=20, overfit_batches=10, limit_train_batches=1, limit_val_batches=0
trainer = Trainer(precision="16-mixed", gradient_clip_val=500, max_time={"hours": 2}, limit_val_batches=1, callbacks=[
                  trainingmode], accumulate_grad_batches=1, check_val_every_n_epoch=1, reload_dataloaders_every_n_epochs=1000000, logger=wandb_logger)
# logger=wandb_logger,

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(limit_val_batches=1)` was configured so 1 batch will be used.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(limit_val_batches=1)` was configured so 1 batch will be used.


In [14]:
# sanity. should be overfiting.
trainer = Trainer(precision="16-mixed", gradient_clip_val=500, max_time={"hours": 2}, limit_val_batches=1, callbacks=[
                  trainingmode], accumulate_grad_batches=1, check_val_every_n_epoch=1, overfit_batches=1, reload_dataloaders_every_n_epochs=1000_000, logger=wandb_logger, num_sanity_val_steps=0)

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(overfit_batches=1)` was configured so 1 batch will be used.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(overfit_batches=1)` was configured so 1 batch will be used.


In [16]:
from scprint.tasks.cell_emb import default_benchmark

In [9]:
model.lr_reduce_patience = 50
model.lr_reduce_ratio = 0.2

In [10]:
model.lr = 0.00001

In [17]:
default_benchmark(model, default_dataset="lung")

/home/ml4ig1/miniconda3/envs/scprint/lib/python3.10/site-packages/anndata/__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
/home/ml4ig1/miniconda3/envs/scprint/lib/python3.10/site-packages/anndata/_io/specs/registry.py:249: OldFormatWarning: Element '/layers' was written without encoding metadata.
  return self.callback(read_func, elem.name, elem, iospec=get_spec(elem))
/home/ml4ig1/miniconda3/envs/scprint/lib/python3.10/site-packages/anndata/_io/utils.py:205: OldFormatWarning: Element '/obs/__categories/dataset' was written without encoding metadata.
  return func(*args, **kwargs)
/home/ml4ig1/miniconda3/envs/scprint/lib/python3.10/site-packages/anndata/_io/utils.py:205: OldFormatWarning: Element '/obs/dataset' was written without encoding metadata.
  return func(*args, **kwargs)
/home/ml4ig1/miniconda3/envs/scprint/lib/python3.10/site-packages/anndata/_io/utils.py:205: OldFormatWarnin

X was not raw counts, using 'counts' layer
Dropping layers:  KeysView(Layers with keys: counts)
checking raw counts
Data is not raw counts, please check layers, find raw data, or bypass with force_preprocess
removed 0 non primary cells, 32472 renamining
filtered out 221 cells, 32251 renamining
Removed 0 genes.
startin QC
Seeing 10331 outliers (32.03% of total dataset):
done
> /home/ml4ig1/Documents code/scPRINT/scprint/tasks/cell_emb.py(383)default_benchmark()
    381     import pdb
    382     pdb.set_trace()
--> 383     embedder = Embedder(
    384         model, pred_embedding=["cell_type_ontology_term_id"], organisms=[adata.obs["organism_ontology_term_id"].values[0]]
    385     )



Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/ml4ig1/miniconda3/envs/scprint/lib/python3.10/site-packages/torch/utils/tensorboard/__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, "__version__") or LooseVersion(
/home/ml4ig1/miniconda3/envs/scprint/lib/python3.10/site-packages/torch/utils/tensorboard/__init__.py:6: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  ) < LooseVersion("1.15"):
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 0it [00:00, ?it/s]

ValueError: 'cell_type_ontology_term_id' is not in list

In [15]:
trainer.fit(model, dat
amodule=datamodule)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type                    | Params
----------------------------------------------------------
0 | gene_encoder  | GeneEncoder             | 11.9 M
1 | expr_encoder  | ContinuousValueEncoder  | 264 K 
2 | pos_encoder   | PositionalEncoding      | 0     
3 | class_encoder | CategoryValueEncoder    | 512   
4 | depth_encoder | ContinuousValueEncoder  | 264 K 
5 | transformer   | FlashTransformerEncoder | 3.2 M 
6 | expr_decoder  | ExprDecoder             | 528 K 
7 | cls_decoders  | ModuleDict              | 0     
8 | mvc_decoder   | MVCDecoder              | 1.1 M 
----------------------------------------------------------
5.3 M     Trainable params
11.9 M    Non-trainable params
17.1 M    Total params
68.459    Total estimated model params size (MB)
wandb: WARNING Serializing object of type list that is 194680 bytes
wandb: WARNING Serializing object of type list that is 194680 bytes
wandb: WARNING Serializing object of type l

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

AnnData object with n_obs × n_vars = 1 × 512
couldn't log to tensorboard
couldn't log to wandb


/home/ml4ig1/miniconda3/envs/scprint/lib/python3.10/site-packages/anndata/__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
/home/ml4ig1/miniconda3/envs/scprint/lib/python3.10/site-packages/anndata/_io/specs/registry.py:249: OldFormatWarning: Element '/layers' was written without encoding metadata.
  return self.callback(read_func, elem.name, elem, iospec=get_spec(elem))
/home/ml4ig1/miniconda3/envs/scprint/lib/python3.10/site-packages/anndata/_io/utils.py:205: OldFormatWarning: Element '/obs/__categories/dataset' was written without encoding metadata.
  return func(*args, **kwargs)
/home/ml4ig1/miniconda3/envs/scprint/lib/python3.10/site-packages/anndata/_io/utils.py:205: OldFormatWarning: Element '/obs/dataset' was written without encoding metadata.
  return func(*args, **kwargs)
/home/ml4ig1/miniconda3/envs/scprint/lib/python3.10/site-packages/anndata/_io/utils.py:205: OldFormatWarnin

X was not raw counts, using 'counts' layer
Dropping layers:  KeysView(Layers with keys: counts)
checking raw counts
Data is not raw counts, please check layers, find raw data, or bypass with force_preprocess
removed 0 non primary cells, 32472 renamining
filtered out 221 cells, 32251 renamining
Removed 0 genes.
startin QC
Seeing 10331 outliers (32.03% of total dataset):
done
> /home/ml4ig1/Documents code/scPRINT/scprint/tasks/cell_emb.py(383)default_benchmark()
    381     import pdb
    382     pdb.set_trace()
--> 383     embedder = Embedder(
    384         model, pred_embedding=["cell_type_ontology_term_id"]
    385     )  # ), 'sex_ontology_term_id', "disease_ontology_term_id"])

> /home/ml4ig1/Documents code/scPRINT/scprint/tasks/cell_emb.py(384)default_benchmark()
    382     pdb.set_trace()
    383     embedder = Embedder(
--> 384         model, pred_embedding=["cell_type_ontology_term_id"]
    385     )  # ), 'sex_ontology_term_id', "disease_ontology_term_id"])
    386     embed

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


--Return--
None
> /home/ml4ig1/Documents code/scPRINT/scprint/tasks/cell_emb.py(82)__init__()
     80         self.doplot = doplot
     81         self.model.doplot = doplot
---> 82         self.trainer = Trainer(precision=precision)
     83         # subset_hvg=1000, use_layer='counts', is_symbol=True,force_preprocess=True, skip_validate=True)
     84 

> /home/ml4ig1/Documents code/scPRINT/scprint/tasks/cell_emb.py(386)default_benchmark()
    384         model, pred_embedding=["cell_type_ontology_term_id"]
    385     )  # ), 'sex_ontology_term_id', "disease_ontology_term_id"])
--> 386     embed_adata, metrics = embedder(adata.copy())
    387     # if do_class and default_dataset == "lung":
    388 

IndexError: index 0 is out of bounds for axis 0 with size 0
> /home/ml4ig1/Documents code/scPRINT/scprint/tasks/cell_emb.py(386)default_benchmark()
    384         model, pred_embedding=["cell_type_ontology_term_id"]
    385     )  # ), 'sex_ontology_term_id', "disease_ontology_term_id"]

IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
expression[:100]

In [ ]:
weirdly, when input expression is set to 0 and pi is set to 1000. I get a loss of 0
when input expression is set to some set of values and pi=-10, disp=max, mean=input, I get 0.479, decreasing pi doesn't help, making it higher, makes it worse, max of disp is ~3M afterward it increases back (likely due to some overflow??). I am guessing that it would need to be set to infinity for it to work. might be better to use dispersion instead of inverse dispersion?

all things equal, if disp is set to 2_700_000, I get loss tensor(3.8694, device='cuda:0')
if disp is 2_700_000+1 -> tensor(0.1994, device='cuda:0')

basically it seems that at too high values or at too low values we get strong instabilities that are exacerbated by the fact that we are in float16.
could it also be because of the espilon? nope

-> why doesn't my model go to this low loss value??



In [ ]:
1. there seems to be an in-between value of disp that makes sense given the expression data
2. 

In [ ]:
import torch.nn.functional as F

In [ ]:
F.mse_loss(expression, expression, reduction="mean")

In [ ]:
loss.nb(
    theta=disp,
    mu=expression,
    target=expression,
)

In [ ]:
loss.zinb(
    theta=output["disp"],
    pi=output["zero_logits"],
    mu=output["mean"],
    target=expression,
)

In [ ]:
loss.zinb(
    theta=output["disp"] + 10000,
    pi=output["zero_logits"]-3,
    mu=expression,
    target=expression,
)

In [ ]:
loss.nb(
    theta=output["disp"],
    mu=output["mean"],
    target=expression,
).mean()

In [ ]:
expr = expression.clone()
expr[:,-20:] = 0
expr[:,:20] +=1

In [ ]:
loss.zinb(
    theta=disp,
    pi=pi,
    mu=output["mean"],
    target=expression,
)

In [ ]:
loss.zinb(
    theta=disp,
    pi=pi,
    mu=1e4*output["mean"]/expression.sum(),
    target=1e4*expression/expression.sum(),
)

In [ ]:
loss.zinb(
    theta=disp,
    pi=pi,
    mu=1e4*expr/expression.sum(),
    target=1e4*expression/expression.sum(),
)

In [ ]:
exp(15)

In [ ]:
disp = torch.zeros_like(output["disp"])
pi = torch.zeros_like(output["disp"]) - 10

In [ ]:
# why if it is the inverse dispersion? does setting it to 3_000_000 works but not 10_000_000?
# maybe some overflow??

In [ ]:
loss.zinb(
    theta=disp,
    pi=pi,
    mu=output["mean"],
    target=expression,
)

In [ ]:
loss.zinb(
    theta=disp,
    pi=pi,
    mu=expression,
    target=expression,
)

In [ ]:
loss.zinb(
    theta=output["disp"],
    pi=output["zero_logits"],
    mu=expression,
    target=expression,
)

In [ ]:
loss.zinb(
    theta=disp,
    pi=pi,
    mu=expression,
    target=expression,
)

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir="../data/tensorboard"

In [ ]:
# wandb_logger.finalize(status="aborted")
torch.cuda.empty_cache()